# Installation


In [ ]:
%pip install -r requirements.txt

# Imports

In [1]:
import os
from os.path import join
import random
from tqdm import tqdm
from glob import glob
from pdf2image import convert_from_path
from PIL import Image

# Paths

In [2]:
data_dir = "/Data"

# Pdf to images

In [ ]:
def convert_pdf_to_images(pdf_path, output_dir):